# Кластеризация для оптимального размещения баннеров с помощью MeanShift
В качестве входных данных выступают географические координаты точек засвета людей в соцсетях, а также координаты офисов рекламного агенства.
Необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов и на основе них выбрать те центры, которые наименее удалены от офисов для того, чтобы компания могла разместить баннеры на улицах с меньшими тратами.
Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров. Таким алгоритмом представляется MeanShif.
Используем его для решения задачи

Задача представлена по ссылке  https://www.coursera.org/learn/unsupervised-learning/programming/XXda2/razmieshchieniie-bannierov

In [1]:
# Импортируем библиотеки
import pandas as pd
import numpy as np

In [2]:
## Считываем данные и берем первые 100000 записей для быстроты расчета центров кластеров
df = pd.read_csv('chekins.csv', sep = ';')
df = df.head(100000)

In [3]:
# Удаляем ненужные поля
df.drop(['id', 'user_id','venue_id',  'created_at'], axis=1 , inplace = True)

In [4]:
df.head()

,latitude,longitude
0,38.895112,-77.036366
1,33.800745,-84.410520
2,45.523452,-122.676207
3,40.764462,-111.904565
4,33.448377,-112.074037


In [5]:
import numpy as np
from sklearn.cluster import MeanShift, estimate_bandwidth
from sklearn.datasets.samples_generator import make_blobs


# Кластеризуем данные с помощью алгоритма MeanShift(не требует задания числа кластеров и его можно 
# ограничить в размерах(заключить в epsilon -окрестность))
X= df.copy()
# The following bandwidth can be automatically detected using
bandwidth = 0.1

ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
ms.fit(X)
labels = ms.labels_
cluster_centers = ms.cluster_centers_

labels_unique = np.unique(labels)
n_clusters_ = len(labels_unique)

print("number of estimated clusters : %d" % n_clusters_)


number of estimated clusters : 3090


In [6]:
# полученные метки кластеров
ms.labels_
df['label']= ms.labels_

In [7]:
#считаем номера кластеров, в которые попадают более 15 наблюдений и записываем их в массив indexes
indexes= df.groupby(['label']).size()[df.groupby(['label']).size()>15].index.values

In [8]:
# создаем таблицу с координатами центров кластеров а также их меткой
df2 = pd.DataFrame(ms.cluster_centers_, columns=['longitude', 'lattitude'])
df2.reset_index(inplace=True)
df2.rename(index=str, columns={"index": "label"}, inplace=True)

In [9]:
# выбираем координаты тех центров, которые содержат больше 15 элементов
df3 = df2.loc[df2['label'].isin(indexes)]

In [10]:
df3.head()

,label,longitude,lattitude
0,0,40.717716,-73.991835
1,1,33.446380,-111.901888
2,2,33.448410,-112.074004
3,3,41.878244,-87.629843
4,4,37.688682,-122.409330


In [171]:
# Считываем данные по офисам компании(с их координатами)
df_coord= pd.read_csv('office.txt', sep =',')

In [172]:
# Используем формулу гаверсинуса для того, чтобы оценить расстояние между двумя точками 
from math import radians, cos, sin, asin, sqrt
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    # Radius of earth in kilometers is 6371
    km = 6371* c
    return km

In [173]:
# Для каждого центра кластера(центра скоплений) расчитываем расстояние до всех офисов компании и записываем 
# результат в массив dist_list
dist_list = np.zeros((df3.shape[0], df_coord.shape[0]))

for rows1 in range(df3.shape[0]):
    lon1 = df3.iloc[rows1].tolist()[1]
    lat1 = df3.iloc[rows1].tolist()[2]
    for rows2 in range(df_coord.shape[0]):
        dist=0.0
        lon2 = df_coord.loc[rows2][0]
        lat2 = df_coord.loc[rows2][1]
        dist= haversine(lon1, lat1, lon2, lat2)
        dist_list[rows1][rows2] = dist  

In [174]:
# Каждая колонка- расстояние до офисов от центра кластера(строка- центр кластера, столбцы - номер офиса)
distance_df= pd.DataFrame(dist_list, columns=['frst_cntr_to_shop_dist', 'scnd_cntr_to_shop_dist', 'third_cntr_to_shop_dist',\
                                 'fourth_cntr_to_shop_dist', 'fifth_cntr_to_shop_dist', 'sixth_cntr_to_shop_dist'])

In [175]:
# Минимальное расстояние от центра кластера до офиса наблюдается рядом с 6-м офисом
distance_df.min()

frst_cntr_to_shop_dist        5.391081
scnd_cntr_to_shop_dist        1.833894
third_cntr_to_shop_dist       5.566228
fourth_cntr_to_shop_dist      1.038522
fifth_cntr_to_shop_dist     661.562854
sixth_cntr_to_shop_dist       0.763703
dtype: float64

In [179]:
# Находим индекс, на котором наблюдаеся наименьшее расстояние до ближайшего центра
distance_df.sixth_cntr_to_shop_dist[distance_df.sixth_cntr_to_shop_dist ==distance_df.sixth_cntr_to_shop_dist.min()]

401    0.763703
Name: sixth_cntr_to_shop_dist, dtype: float64

In [ ]:
# Находим координаты ближайшего центра кластера к одному из офисов,  где можно разместить баннер
df3.iloc[401]

## Визуализация

In [44]:
# Для визуализации координат на карте используем библиотеку folium(для её установки необходимо прописать pip install folium)
import os
import folium
print(folium.__version__)

0.5.0


In [45]:
# Из исходных данных берем те координаты людей, которые попадают в кластеры, содержащие более 15 абонентов в зоне покрытия
new = df.loc[df['label'].isin(indexes)]
new.head()

,latitude,longitude,label
0,38.895112,-77.036366,5
1,33.800745,-84.410520,7
2,45.523452,-122.676207,28
3,40.764462,-111.904565,63
4,33.448377,-112.074037,2


In [48]:
## Каждому пользователю сети присваиваем вес = количество абонентов в его кластере / общее количество пользователей сети
new1 = new.groupby(['label']).size()/new.shape[0]
new1 = new1.reset_index()
new1.columns = ['label', 'weights']

res_df = pd.merge(new, new1, how = 'left', on = 'label')
res_df.drop(['label'],axis=1,  inplace=True)
res_df = res_df.as_matrix().tolist()

In [55]:
## Визуализация весов для координат каждого абонента(так как количество координат больше, сохраним тепловую карту в html формате)
from folium.plugins import HeatMap

m = folium.Map([38., -77.], tiles='stamentoner', zoom_start=6)

HeatMap(res_df).add_to(m)

m.save('Heatmap.html')